# Dithered Exposure Series



In [ ]:
# First to connect to the camera

# CAMERA SERVER
from wsp.viscam.pirt_client import CameraClient

# Connect to the camera server
cam = CameraClient("192.168.1.15", 5555)
cam.connect()

# Make sure the camera has all corrections turned off
cam.set_correction("GAIN", "OFF")
cam.set_correction("OFFSET", "OFF")   
cam.set_correction("SUB", "OFF")
cam.set_observer_name("Nate Lourie")


Connected to camera server at 192.168.1.15:5555


{'status': 'success', 'message': 'Object set to VIS-SWIR QE Measurement'}

In [30]:
# Get set up to communicate with WSP
import os
import time
from datetime import datetime, timedelta
import numpy as np
import sys
from socket import socket, AF_INET, SOCK_STREAM

#%% init a tcp-ip connection to wsp

sock = socket(AF_INET, SOCK_STREAM)
wsp_cmd_server_address = ("192.168.1.10", 7000)
sock.connect(wsp_cmd_server_address)

def send(cmd):
    try:
        sock.sendall(bytes(cmd, "utf-8"))
        return 1
    except Exception as e:
        return 0, e

# test the comman connection
send("xyzzy")

1

1

In [13]:
from astropy.coordinates import Angle
import astropy.units as u

# RA/Dec of the target
raDeg, decDeg = 233.468548,  30.833366

ra, dec = "15:33:52.45", "+30:50:00.12"

ra_astropy = Angle(ra, unit = u.hour)
dec_astropy = Angle(dec, unit = u.deg)

print(f"RA: converted RA = {ra_astropy.deg:.4f} deg, Fritz raDeg = {raDeg:.4f} deg")
print(f"Dec: converted Dec = {dec_astropy.deg:.4f} deg, Fritz decDeg = {decDeg:.4f} deg")

targName = "WNTR25fboyy"
filterName = "J"
progPI = "Lourie"

#%% Set up the exposure series parameters
n_dithers = 8
exptime = 30.0
n_imgs_per_dither = 4
# note that this is the same total exptime
# as the WINTER 8*120 s approach but with shorter
# exposures

#%% Set up the dither pattern
ditherMinStep_as = 75 # ditherMinStep_as
ditherMaxStep_as = 90 # ditherMaxStep_as




RA: converted RA = 233.4685 deg, Fritz raDeg = 233.4685 deg
Dec: converted Dec = 30.8334 deg, Fritz decDeg = 30.8334 deg


In [ ]:
# Get on target

send("dome_tracking_on")
time.sleep(1)
cmd = f"mount_goto_ra_dec_j2000 {ra} {dec}"
send(cmd)


In [16]:
# Need to apply any offsets?

ra_offset_arcmin = 0.0
dec_offset_arcmin = 0.0

# apply the offsets in arcseconds
ra_offset_arcsec = ra_offset_arcmin * 60.0
dec_offset_arcsec = dec_offset_arcmin * 60.0
if ra_offset_arcsec !=0.0:
    send(f"mount_offset ra add_arcsec {ra_offset_arcsec:.0f}")
if dec_offset_arcsec !=0.0:
    send(f"mount_offset dec add_arcsec {dec_offset_arcsec:.0f}")




In [ ]:
# send some commands to the camera
cam.set_object_name(targName)

cam.set_exposure(exptime)

cam.set_save_path(f"~\data\commissioning\20250803\{targName}")


{'status': 'success',
 'message': 'Save path set to C:\\Users\\oir-user\\data\\commissioning\x8250803\\{targName}'}

In [1]:
# Execute the dithered exposure series

print(f"Starting dithered exposure series for target {targName} at RA {ra} and Dec {dec}")
print(f"Executing {n_dithers} dithers with {n_imgs_per_dither} images at {exptime} s each")

for i in range(n_dithers):

    print(f"Starting dither {i+1} of {n_dithers}"
          f" with {n_imgs_per_dither} images at {exptime} s each"
    )

    # build the header
    header = {
        "TARGNAME": targName,
        "OBSTYPE" : "SCIENCE",
        "DATE-OBS": datetime.utcnow().strftime("%Y%m%d %H%M%S.%f"),
        "OBJECT": targName,
        "FILTER": filterName,
        "PROGPI": progPI,
        "RADEG": raDeg,
        "DECDEG": decDeg,
        "OBJRA": raDeg,
        "OBJDEC": decDeg,
        "REXPTIME": exptime,
        "n_imgs_per_dither": n_imgs_per_dither,
        "DITHNUM": i,
        "NUMDITHS": n_dithers,
        "DITHSTEP": ditherMaxStep_as,
    }

    # Assume we have already gotten on target, so fist dither just start taking images
    if i == 0:
        print("Starting first dither, no dither offset applied")

    
        # take n_imgs_per_dither images
        cam.capture_frames(n_imgs_per_dither, stack = False, headers = header, show_progress = False)
        
    else:
        # apply a dither offset

        radius = np.random.uniform(ditherMinStep_as, ditherMaxStep_as)
        theta = np.random.uniform(0, np.pi)
        ra_dither_arcsec = radius * np.cos(theta)
        dec_dither_arcsec = radius * np.sin(theta)

        msg = f"Dither (dRA, dDec) = ({ra_dither_arcsec:0.1f}, {dec_dither_arcsec:.1f}) as"
        print(msg)
        send(
            f"mount_dither_arcsec_radec {ra_dither_arcsec} {dec_dither_arcsec}"
        )

        # give it a little time to settle
        settle_time = 10
        print(f"Settling for {settle_time} seconds for mount to settle")
        time.sleep(settle_time)

        # take n_imgs_per_dither images
        cam.capture_frames(n_imgs_per_dither, stack = False, headers = header, show_progress = False)

print("Dithered exposure series complete!")

NameError: name 'targName' is not defined